In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_csv("spotify_2500_refined_moods (1).csv")

# 1. Print info and head for initial inspection
print(df.info())  # Check data types and missing values
print(df.head())  # Preview the first few rows

# 2. Focus on the problematic columns
print(df[['clean_lyrics', 'lyrics_sentiment', 'valence', 'energy', 'danceability', 'tempo']].head())
# Check for any unusual characters or formats

# 3. Try loading the dataset again before this cell to isolate the issue
df = pd.read_csv("spotify_2500_refined_moods (1).csv")

# Drop rows with missing values just in case
df = df.dropna(subset=['clean_lyrics', 'lyrics_sentiment', 'valence', 'energy', 'danceability', 'tempo'])

# Define target columns
audio_target = 'mood'
lyrics_target = 'lyrics_sentiment'


# Train-test split (shared for both models)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df[lyrics_target])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   track_id                  2500 non-null   object 
 1   track_name                2499 non-null   object 
 2   track_artist              2499 non-null   object 
 3   track_popularity          2500 non-null   float64
 4   track_album_id            2500 non-null   object 
 5   track_album_name          2499 non-null   object 
 6   track_album_release_date  2500 non-null   object 
 7   playlist_name             2500 non-null   object 
 8   playlist_id               2500 non-null   object 
 9   playlist_genre            2500 non-null   object 
 10  playlist_subgenre         2500 non-null   object 
 11  danceability              2500 non-null   float64
 12  energy                    2500 non-null   float64
 13  key                       2500 non-null   float64
 14  loudness

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_csv("spotify_2500_refined_moods (1).csv")

# 🧼 Clean mood labels (strip spaces, lowercase)
df['mood'] = df['mood'].astype(str).str.strip().str.lower()

# 🔍 See class distribution
print("Mood distribution:\n", df['mood'].value_counts())

# 🎯 Audio features to use
audio_features = ['valence', 'energy', 'danceability', 'tempo']
X = df[audio_features]
y = df['mood']

# ✂️ Split into 80% train, 20% test (with stratification)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # preserve class proportions
)

# 🔢 Encode labels
label_encoder = LabelEncoder()
y_train_enc = label_encoder.fit_transform(y_train)
y_test_enc = label_encoder.transform(y_test)

# 🌳 Train Random Forest
model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
model.fit(X_train, y_train_enc)

# 🧪 Predict on the test set
y_pred = model.predict(X_test)

# 📊 Evaluate
print("\nClassification Report on 50/50 Train-Test Split:")
print(classification_report(y_test_enc, y_pred, target_names=label_encoder.classes_))


Mood distribution:
 mood
calm           1048
energetic       483
intense         319
melancholic     280
joyful          186
relaxed         107
sad              59
hype             18
Name: count, dtype: int64

Classification Report on 50/50 Train-Test Split:
              precision    recall  f1-score   support

        calm       0.96      0.94      0.95       209
   energetic       0.85      0.96      0.90        97
        hype       0.00      0.00      0.00         4
     intense       0.82      0.92      0.87        64
      joyful       0.82      0.89      0.86        37
 melancholic       0.73      0.71      0.72        56
     relaxed       0.56      0.24      0.33        21
         sad       0.70      0.58      0.64        12

    accuracy                           0.87       500
   macro avg       0.68      0.66      0.66       500
weighted avg       0.85      0.87      0.86       500



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import joblib

# Save
joblib.dump(model, 'rf_model.pkl')
joblib.dump(label_encoder, 'mood_encoder.pkl')

# Load (in Visual Studio or any Python script)
rf_model = joblib.load('rf_model.pkl')
mood_encoder = joblib.load('mood_encoder.pkl')


In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Load and prepare dataset
df = pd.read_csv("spotify_2500_refined_moods (1).csv")
df = df.dropna(subset=['clean_lyrics', 'lyrics_sentiment'])
df['lyrics_sentiment'] = df['lyrics_sentiment'].str.strip().str.lower()

# Optional: Merge "surprise" into "joy"
df['lyrics_sentiment'] = df['lyrics_sentiment'].replace({'surprise': 'joy'})

# Encode labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['lyrics_sentiment'])
num_labels = len(label_encoder.classes_)

# Split dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['clean_lyrics'], df['label'], test_size=0.2, stratify=df['label'], random_state=42
)

# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_enc = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128, return_tensors='pt')
test_enc = tokenizer(list(test_texts), truncation=True, padding=True, max_length=128, return_tensors='pt')

# Dataset class
class LyricsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

train_dataset = LyricsDataset(train_enc, train_labels.to_numpy())
test_dataset = LyricsDataset(test_enc, test_labels.to_numpy())

# Load model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

# Add dropout to help regularization
model.bert.dropout = nn.Dropout(0.3)

# Move to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(df['label']), y=df['label'])
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Loss + optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = AdamW(model.parameters(), lr=2e-5)

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

# Training loop
epochs = 6
for epoch in range(epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = criterion(outputs.logits, batch['labels'])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        preds = torch.argmax(outputs.logits, dim=1)
        total_loss += loss.item()
        correct += (preds == batch['labels']).sum().item()
        total += batch['labels'].size(0)

    train_acc = correct / total
    avg_loss = total_loss / len(train_loader)

    # Evaluate on test set during training
    model.eval()
    test_preds, test_labels_actual = [], []
    with torch.no_grad():
        for batch in test_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            preds = torch.argmax(outputs.logits, dim=1)
            test_preds.extend(preds.cpu().numpy())
            test_labels_actual.extend(batch['labels'].cpu().numpy())

    test_acc = accuracy_score(test_labels_actual, test_preds)

    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")

# Final Report
print("\n📊 Final Classification Report:")
print(classification_report(
    test_labels_actual,
    test_preds,
    labels=list(range(num_labels)),
    target_names=label_encoder.classes_
))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/6 | Train Loss: 1.5836 | Train Acc: 0.3505 | Test Acc: 0.4538
Epoch 2/6 | Train Loss: 1.5172 | Train Acc: 0.4058 | Test Acc: 0.4699
Epoch 3/6 | Train Loss: 1.2875 | Train Acc: 0.5710 | Test Acc: 0.5261
Epoch 4/6 | Train Loss: 0.8202 | Train Acc: 0.7271 | Test Acc: 0.5944
Epoch 5/6 | Train Loss: 0.3654 | Train Acc: 0.8751 | Test Acc: 0.6024
Epoch 6/6 | Train Loss: 0.1662 | Train Acc: 0.9486 | Test Acc: 0.6988

📊 Final Classification Report:
              precision    recall  f1-score   support

       anger       0.67      0.81      0.73        83
        fear       0.45      0.75      0.56        12
         joy       0.77      0.67      0.72        87
        love       0.42      0.67      0.52        12
     sadness       0.91      0.58      0.71        55

    accuracy                           0.70       249
   macro avg       0.65      0.69      0.65       249
weighted avg       0.74      0.70      0.70       249



In [ ]:
from sklearn.metrics import classification_report

# Run prediction
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, axis=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(inputs['labels'].cpu().numpy())

# Fix: Ensure all label indices are included
labels = list(range(len(label_encoder.classes_)))  # e.g., [0, 1, 2, 3, 4, 5]

# Display classification report
print("\n📊 Classification Report (BERT Lyrics Sentiment Model):")
print(classification_report(
    all_labels,
    all_preds,
    labels=labels,
    target_names=label_encoder.classes_
))



📊 Classification Report (BERT Lyrics Sentiment Model):
              precision    recall  f1-score   support

       anger       0.67      0.81      0.73        83
        fear       0.45      0.75      0.56        12
         joy       0.77      0.67      0.72        87
        love       0.42      0.67      0.52        12
     sadness       0.91      0.58      0.71        55

    accuracy                           0.70       249
   macro avg       0.65      0.69      0.65       249
weighted avg       0.74      0.70      0.70       249



In [ ]:
# === Save BERT model weights ===
torch.save(model.state_dict(), "bert_sentiment_model.pt")
print("✅ BERT model weights saved to 'bert_sentiment_model.pt'")

# === Save the label encoder (important for inference) ===
import joblib
joblib.dump(label_encoder, "bert_sentiment_label_encoder.pkl")
print("✅ Label encoder saved to 'bert_sentiment_label_encoder.pkl'")


✅ BERT model weights saved to 'bert_sentiment_model.pt'
✅ Label encoder saved to 'bert_sentiment_label_encoder.pkl'


In [ ]:
# Save
import joblib

# 1. Save the trained BERT model
model.save_pretrained("bert_sentiment_model")

# 2. Save the tokenizer
tokenizer.save_pretrained("bert_sentiment_model")

# 3. Save the label encoder (class-to-index mapping)
joblib.dump(label_encoder, "bert_label_encoder.pkl")

print("✅ Model, tokenizer, and label encoder saved successfully.")


✅ Model, tokenizer, and label encoder saved successfully.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Path where you want to save the model in your Google Drive
model_save_path = '/content/drive/MyDrive/bert_sentiment_model.pt'

# Save the model
torch.save(model.state_dict(), model_save_path)
print(f"✅ BERT model saved to: {model_save_path}")


✅ BERT model saved to: /content/drive/MyDrive/bert_sentiment_model.pt


In [ ]:
def predict_sentiment(text, model, tokenizer, label_encoder):
    model.eval()

    # Tokenize the new text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class_id = torch.argmax(logits, dim=1).item()

    predicted_label = label_encoder.inverse_transform([predicted_class_id])[0]
    return predicted_label


In [ ]:
sample_lyric_1 = "I'm walking on sunshine and I feel so happy!"
sample_lyric_2 = "I’m broken inside, everything feels cold and dark."
sample_lyric_3 = "I have no idea what’s going on, just confused."

print("Test 1:", predict_sentiment(sample_lyric_1, model, tokenizer, label_encoder))
print("Test 2:", predict_sentiment(sample_lyric_2, model, tokenizer, label_encoder))
print("Test 3:", predict_sentiment(sample_lyric_3, model, tokenizer, label_encoder))


Test 1: joy
Test 2: sadness
Test 3: sadness


In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Load dataset
df = pd.read_csv("spotify_2500_refined_moods (1).csv")
df = df.dropna(subset=['clean_lyrics', 'lyrics_sentiment', 'mood'])

# Clean and encode labels
df['lyrics_sentiment'] = df['lyrics_sentiment'].str.strip().str.lower()
df['mood'] = df['mood'].str.strip().str.lower()

sentiment_encoder = LabelEncoder()
df['sentiment_label'] = sentiment_encoder.fit_transform(df['lyrics_sentiment'])

mood_encoder = LabelEncoder()
df['mood_label'] = mood_encoder.fit_transform(df['mood'])

# Audio features
audio_features = ['valence', 'energy', 'danceability', 'tempo']

# Train/test split
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['mood_label'], random_state=42)

# Re-train the random forest on training set
rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
rf_model.fit(train_df[audio_features], train_df['mood_label'])

# Load trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(sentiment_encoder.classes_))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)
bert_model.eval()

# Function to get BERT sentiment probabilities
def get_bert_probs(texts):
    inputs = tokenizer(list(texts), padding=True, truncation=True, return_tensors="pt", max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = bert_model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1).cpu().numpy()
    return probs

# Build dataset of combined predictions
fusion_inputs = []
fusion_labels = []

for _, row in test_df.iterrows():
    audio_vec = rf_model.predict_proba([row[audio_features].values])[0]
    lyrics_vec = get_bert_probs([row['clean_lyrics']])[0]
    combined_vec = np.concatenate([audio_vec, lyrics_vec])
    fusion_inputs.append(combined_vec)
    fusion_labels.append(row['mood_label'])

X_fusion = np.array(fusion_inputs)
y_fusion = np.array(fusion_labels)

# Dataset and DataLoader for FNN
class FusionDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = torch.tensor(inputs, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.inputs[idx], self.labels[idx]

fusion_dataset = FusionDataset(X_fusion, y_fusion)
fusion_loader = DataLoader(fusion_dataset, batch_size=16, shuffle=True)

# Define the FusionNet model
class FusionNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FusionNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

fusion_model = FusionNet(input_dim=X_fusion.shape[1], hidden_dim=32, output_dim=len(mood_encoder.classes_))
fusion_model.to(device)

# TRAINING: FusionNet for 20 Epochs

optimizer = torch.optim.Adam(fusion_model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
epochs = 20

for epoch in range(epochs):
    fusion_model.train()
    total_loss, correct, total = 0, 0, 0
    for features, labels in fusion_loader:
        features, labels = features.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = fusion_model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    acc = correct / total
    avg_loss = total_loss / len(fusion_loader)
    print(f"Epoch {epoch+1:02d} | Loss: {avg_loss:.4f} | Accuracy: {acc:.4f}")





Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feat

Epoch 01 | Loss: 2.1440 | Accuracy: 0.1124
Epoch 02 | Loss: 2.0474 | Accuracy: 0.1767
Epoch 03 | Loss: 1.9541 | Accuracy: 0.3574
Epoch 04 | Loss: 1.8558 | Accuracy: 0.6867
Epoch 05 | Loss: 1.7749 | Accuracy: 0.7269
Epoch 06 | Loss: 1.6677 | Accuracy: 0.7068
Epoch 07 | Loss: 1.5689 | Accuracy: 0.7229
Epoch 08 | Loss: 1.4810 | Accuracy: 0.7108
Epoch 09 | Loss: 1.3617 | Accuracy: 0.7390
Epoch 10 | Loss: 1.2875 | Accuracy: 0.7189
Epoch 11 | Loss: 1.2022 | Accuracy: 0.7149
Epoch 12 | Loss: 1.1094 | Accuracy: 0.7390
Epoch 13 | Loss: 1.0546 | Accuracy: 0.7510
Epoch 14 | Loss: 0.9892 | Accuracy: 0.7590
Epoch 15 | Loss: 0.9036 | Accuracy: 0.7671
Epoch 16 | Loss: 0.8533 | Accuracy: 0.7711
Epoch 17 | Loss: 0.8297 | Accuracy: 0.7791
Epoch 18 | Loss: 0.7698 | Accuracy: 0.7912
Epoch 19 | Loss: 0.7501 | Accuracy: 0.7912
Epoch 20 | Loss: 0.6958 | Accuracy: 0.7952


In [ ]:
# Save
torch.save(fusion_model.state_dict(), 'fusion_model.pth')



In [ ]:
from sklearn.metrics import classification_report

fusion_model.eval()
all_preds, all_true = [], []

with torch.no_grad():
    for features, labels in fusion_loader:
        features = features.to(device)
        outputs = fusion_model(features)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_true.extend(labels.numpy())

print("\n📊 Final FusionNet Classification Report:")
print(classification_report(
    all_true,
    all_preds,
    labels=list(range(len(mood_encoder.classes_))),
    target_names=mood_encoder.classes_
))



📊 Final FusionNet Classification Report:
              precision    recall  f1-score   support

        calm       0.79      0.99      0.88       108
   energetic       0.83      1.00      0.91        50
        hype       0.00      0.00      0.00         2
     intense       0.82      1.00      0.90        32
      joyful       1.00      0.17      0.29        18
 melancholic       0.91      0.40      0.56        25
     relaxed       0.00      0.00      0.00        10
         sad       0.00      0.00      0.00         4

    accuracy                           0.81       249
   macro avg       0.54      0.44      0.44       249
weighted avg       0.78      0.81      0.76       249



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
def predict_fusion_mood(lyrics_text, audio_feature_row):
    # Get audio prediction probabilities from trained Random Forest
    audio_probs = rf_model.predict_proba([audio_feature_row])[0]

    # Get BERT sentiment prediction probabilities
    lyrics_probs = get_bert_probs([lyrics_text])[0]

    # Combine both for FusionNet
    fusion_input = np.concatenate([audio_probs, lyrics_probs])
    fusion_tensor = torch.tensor(fusion_input, dtype=torch.float32).unsqueeze(0).to(device)

    # Get FusionNet prediction
    fusion_model.eval()
    with torch.no_grad():
        output = fusion_model(fusion_tensor)
        pred_idx = torch.argmax(output, dim=1).item()
        predicted_mood = mood_encoder.inverse_transform([pred_idx])[0]

    return predicted_mood


In [ ]:
# Sample lyrics (new)
new_lyrics = "Dancing in the sunshine, smiling every day, life feels like a dream, and I’m floating away."

# Simulated Spotify-style audio features (values from 0–1 or real-world ranges)
new_audio_features = {
    'valence': 0.85,           # high positivity
    'energy': 0.78,
    'danceability': 0.7,
    'tempo': 120,
    'acousticness': 0.2,
    'loudness': -5.0,          # dB
    'speechiness': 0.1,
    'instrumentalness': 0.0,
    'liveness': 0.2,
    'mode': 1,
    'key': 5,                  # F major
    'duration_ms': 210000
}
# Convert dictionary to row-like format
audio_row = pd.Series(new_audio_features)[audio_features]

# Predict using your fusion model
predicted_mood = predict_fusion_mood(new_lyrics, audio_row)

print("🆕 NEW SONG TEST")
print(f"📝 Lyrics: {new_lyrics}")
print(f"🎧 Audio Features: {new_audio_features}")
print(f"🤖 Predicted Mood: {predicted_mood}")


🆕 NEW SONG TEST
📝 Lyrics: Dancing in the sunshine, smiling every day, life feels like a dream, and I’m floating away.
🎧 Audio Features: {'valence': 0.85, 'energy': 0.78, 'danceability': 0.7, 'tempo': 120, 'acousticness': 0.2, 'loudness': -5.0, 'speechiness': 0.1, 'instrumentalness': 0.0, 'liveness': 0.2, 'mode': 1, 'key': 5, 'duration_ms': 210000}
🤖 Predicted Mood: energetic


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [ ]:
def infer_mbti_from_mood(mood_distribution):
    mbti_scores = {
        'E': 0, 'I': 0,
        'S': 0, 'N': 0,
        'T': 0, 'F': 0,
        'J': 0, 'P': 0
    }

    trait_map = {
        'joyful':     ['E', 'F', 'P'],
        'energetic':  ['E', 'S', 'J'],
        'calm':       ['I', 'F', 'P'],
        'sad':        ['I', 'F'],
        'melancholic':['I', 'N', 'T'],
        'intense':    ['E', 'T', 'J'],
        'relaxed':    ['P', 'F', 'N'],
        'hype':       ['E', 'S', 'J']
    }

    for mood, weight in mood_distribution.items():
        traits = trait_map.get(mood, [])
        for trait in traits:
            mbti_scores[trait] += weight

    mbti = ''
    mbti += 'E' if mbti_scores['E'] >= mbti_scores['I'] else 'I'
    mbti += 'S' if mbti_scores['S'] >= mbti_scores['N'] else 'N'
    mbti += 'T' if mbti_scores['T'] >= mbti_scores['F'] else 'F'
    mbti += 'J' if mbti_scores['J'] >= mbti_scores['P'] else 'P'

    return mbti


In [ ]:
playlist_moods = {
    'joyful': 0.4,
    'calm': 0.2,
    'intense': 0.2,
    'sad': 0.1,
    'relaxed': 0.1
}

print("Predicted MBTI:", infer_mbti_from_mood(playlist_moods))


Predicted MBTI: ENFP


In [ ]:
!pip install spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.9/268.9 kB 6.2 MB/s eta 0:00:00


In [ ]:
import spotipy
import pandas as pd
import requests
import re
import time
from bs4 import BeautifulSoup
import base64

# --- Spotify API Authorization ---
# **Replace these with your actual client ID and secret from Spotify Developer Dashboard**
SPOTIFY_CLIENT_ID = ''
SPOTIFY_CLIENT_SECRET = ''

def get_token():
    auth_string = SPOTIFY_CLIENT_ID + ":" + SPOTIFY_CLIENT_SECRET
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    response = requests.post(url, headers=headers, data=data)
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    return response.json()["access_token"]

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}
print(get_token())

BQBljUtx6QS1Wo0B5gXvyB60JtYqw-2CkskB4l7VPKGXQSa7iQA02N4kv_DqAmO0OESDMd42Yh5ucmgz7ZrZNbkiFxlKGHDiafkYhMK5RK9T7vjEAYDZ4w1XHtFUYSIU0iApt5qwdm8


In [ ]:
def get_playlist_tracks_only(playlist_url, token):
    playlist_id = playlist_url.split("/")[-1].split("?")[0]  # Extract ID from URL
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    headers = get_auth_header(token)
    params = {"limit": 100}
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    items = response.json()["items"]

    songs = []
    for item in items:
        track = item['track']
        if track and track['id']:
            songs.append({
                "artist": track['artists'][0]['name'],
                "title": track['name'],
                "id": track['id']
            })
    return songs
print(get_token())
token = get_token()
# Assuming you have your token and playlist_url defined:
playlist_url ="https://open.spotify.com/playlist/0nDIXUJbLjOEwPn1Idhz1r?si=1b6a65dbc0764ebb"  # Example
songs = get_playlist_tracks_only(playlist_url, token)  # 🎯 Pass playlist_url, not playlist_id
print(songs)


BQAck656APaJqWJgMJAo7egVvgnacx6rQKWk6ZXuRRU_abJISKYxU05v54anfast_yLr5tKrOE-65UTAVeJXQg1BQWL_pcXEEpKLNduEj5zXrkX0etQAowiQL_ln9zEqRkpt5D-VZcg
[{'artist': 'The Weeknd', 'title': 'Call Out My Name', 'id': '1HCLJRFIChC0yk8aSTWXiC'}, {'artist': 'The Weeknd', 'title': 'I Was Never There', 'id': '1cKHdTo9u0ZymJdPGSh6nq'}, {'artist': 'The Weeknd', 'title': 'Try Me', 'id': '4ppTAJUbNXELZcoUaL90wo'}, {'artist': 'The Weeknd', 'title': 'Often', 'id': '4PhsKqMdgMEUSstTDAmMpg'}, {'artist': 'The Weeknd', 'title': 'Six Feet Under', 'id': '4mU5iXHeLgbR94siF7p1sY'}, {'artist': 'Pink Floyd', 'title': 'Another Brick in the Wall, Pt. 2', 'id': '4gMgiXfqyzZLMhsksGmbQV'}, {'artist': "Guns N' Roses", 'title': "Knockin' On Heaven's Door", 'id': '4JiEyzf0Md7KEFFGWDDdCr'}, {'artist': 'Led Zeppelin', 'title': 'Stairway to Heaven - Remaster', 'id': '5CQ30WqJwcep0pYcV4AMNc'}, {'artist': "Guns N' Roses", 'title': "Don't Cry (Original)", 'id': '2N2yrmodOnVF10mKvItC9P'}, {'artist': 'Pink Floyd', 'title': 'Time', 'id': 

In [ ]:
songs = get_playlist_tracks_only(playlist_url, token)


In [ ]:
import requests
import re
from bs4 import BeautifulSoup

# --- Scrape fallback from songdata.io ---
def scrape_audio_features(artist, title):
    try:
        # Format the search query
        search_query = f"{artist} {title}".replace(' ', '%20')
        url = f"https://songdata.io/search?q={search_query}"

        headers = {
            "User-Agent": "Mozilla/5.0"
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise exception on 4xx/5xx errors

        soup = BeautifulSoup(response.text, 'html.parser')
        card = soup.find('div', class_='card-body')

        if not card:
            return None

        text = card.get_text(separator="\n")
        features = {}
        if 'Danceability' in text:
            features['danceability'] = float(re.findall(r"Danceability: ([\d.]+)", text)[0]) / 100
        if 'Energy' in text:
            features['energy'] = float(re.findall(r"Energy: ([\d.]+)", text)[0]) / 100
        if 'Valence' in text:
            features['valence'] = float(re.findall(r"Valence: ([\d.]+)", text)[0]) / 100
        if 'Tempo' in text:
            features['tempo'] = float(re.findall(r"Tempo: ([\d.]+)", text)[0])

        return features if features else None

    except requests.exceptions.HTTPError as http_err:
        print(f"❌ HTTP error scraping {artist} - {title}: {http_err}")
    except Exception as e:
        print(f"❌ Error scraping {artist} - {title}: {e}")
    return None


In [ ]:
# --- Local + Web match ---
import re

def normalize(text):
    """Lowercase and remove non-alphanumeric characters."""
    text = text.lower()
    text = re.sub(r"[^a-z0-9 ]", "", text)
    return text

# --- Local + Web match ---

def get_audio_features_local_or_scrape(songs, local_csv_path="spotify_songs.csv"):
    local_df = pd.read_csv(local_csv_path)
    enriched_songs = []

    for song in songs:
        artist = normalize(song["artist"])
        title = normalize(song["title"])
        features = {
            "artist": song["artist"],
            "title": song["title"],
            "danceability": None,
            "energy": None,
            "valence": None,
            "tempo": None
        }

        # Try local match
        match = local_df[
            local_df['track_artist'].apply(lambda x: normalize(str(x))).str.contains(artist) &
            local_df['track_name'].apply(lambda x: normalize(str(x))).str.contains(title)
        ]

        if not match.empty:
            row = match.iloc[0]
            print(f"✅ Found locally: {song['artist']} — {song['title']}")
            features.update({
                "danceability": row['danceability'],
                "energy": row['energy'],
                "valence": row['valence'],
                "tempo": row['tempo']
            })
        else:
            print(f"🔍 Scraping: {song['artist']} — {song['title']}")
            scraped = scrape_audio_features(song["artist"], song["title"])
            if scraped:
                print(f"✅ Scraped successfully: {song['artist']} — {song['title']}")
                features.update(scraped)
            else:
                print(f"❌ Could not find features for: {song['artist']} — {song['title']}")

        enriched_songs.append(features)
        time.sleep(1)

    return pd.DataFrame(enriched_songs)

# STEP 3: Match audio features (locally or via scraping)
audio_feature_df = get_audio_features_local_or_scrape(songs)

# STEP 4: Fetch lyrics (inline)
lyrics_list = []
for _, row in audio_feature_df.iterrows():
    try:
        url = f"https://api.lyrics.ovh/v1/{row['artist']}/{row['title']}"
        response = requests.get(url)
        if response.status_code == 200:
            lyrics_list.append(response.json().get('lyrics', None))
        else:
            lyrics_list.append(None)
        time.sleep(0.5)
    except Exception as e:
        print(f"❌ Error fetching lyrics for {row['title']} by {row['artist']}: {e}")
        lyrics_list.append(None)

audio_feature_df['lyrics'] = lyrics_list

# STEP 5: BERT Sentiment Prediction (for valid lyrics only)
bert_sentiment_probs = []
lyrics_available = []

for lyrics in audio_feature_df['lyrics']:
    if lyrics is None or pd.isna(lyrics):
        bert_sentiment_probs.append(None)
        lyrics_available.append(False)
    else:
        try:
            probs = get_bert_probs(pd.Series([lyrics]))[0]
            bert_sentiment_probs.append(probs)
            lyrics_available.append(True)
        except Exception as e:
            print(f"⚠️ BERT failed: {e}")
            bert_sentiment_probs.append(None)
            lyrics_available.append(False)

# STEP 6–7: Mood Prediction using Fusion, Lyrics-only, Audio-only, or Skip
predicted_moods = []
model_used_list = []
valid_rows = []

for idx, row in audio_feature_df.iterrows():
    has_audio = all(pd.notna([row.get(f) for f in ['valence', 'energy', 'danceability', 'tempo']]))
    has_lyrics = lyrics_available[idx]
    lyrics_probs = bert_sentiment_probs[idx] if has_lyrics else None

    try:
        if has_audio and lyrics_probs is not None:
            audio_vec = rf_model.predict_proba(
                pd.DataFrame([row[['valence', 'energy', 'danceability', 'tempo']]])
            )[0]
            fusion_input = np.concatenate([audio_vec, lyrics_probs])
            fusion_tensor = torch.tensor(fusion_input, dtype=torch.float32).unsqueeze(0).to(device)
            with torch.no_grad():
                output = fusion_model(fusion_tensor)
            mood_idx = torch.argmax(output, dim=1).item()
            model_used = "fusion"

        elif has_lyrics:
            mood_idx = np.argmax(lyrics_probs)
            model_used = "lyrics-only"

        elif has_audio:
            audio_vec = rf_model.predict_proba(
                pd.DataFrame([row[['valence', 'energy', 'danceability', 'tempo']]])
            )[0]
            mood_idx = np.argmax(audio_vec)
            model_used = "audio-only"

        else:
            print(f"⚠️ Skipping {row['title']} by {row['artist']} — no audio or lyrics")
            continue

        mood = mood_encoder.inverse_transform([mood_idx])[0]
        predicted_moods.append(mood)
        model_used_list.append(model_used)
        valid_rows.append(row)

    except Exception as e:
        print(f"⚠️ Prediction failed for {row['title']} by {row['artist']}: {e}")
        continue

# STEP 8: Assign moods to valid songs only
final_df = pd.DataFrame(valid_rows).copy()
final_df['predicted_mood'] = predicted_moods
final_df['model_used'] = model_used_list

# STEP 9: Final output
print("\n🎧 Final Real-Time Mood Analysis:")
print(final_df[['artist', 'title', 'predicted_mood', 'model_used']])


✅ Found locally: The Weeknd — Call Out My Name
✅ Found locally: The Weeknd — I Was Never There
✅ Found locally: The Weeknd — Try Me
✅ Found locally: The Weeknd — Often
✅ Found locally: The Weeknd — Six Feet Under
✅ Found locally: Pink Floyd — Another Brick in the Wall, Pt. 2
✅ Found locally: Guns N' Roses — Knockin' On Heaven's Door
✅ Found locally: Led Zeppelin — Stairway to Heaven - Remaster
✅ Found locally: Guns N' Roses — Don't Cry (Original)
✅ Found locally: Pink Floyd — Time
✅ Found locally: Eagles — Hotel California - 2013 Remaster
✅ Found locally: Ariana Grande — One Last Time
✅ Found locally: Calvin Harris — Slide (feat. Frank Ocean & Migos)
✅ Found locally: Childish Gambino — Feels Like Summer
✅ Found locally: Childish Gambino — 3005
✅ Found locally: Frank Ocean — Nights
✅ Found locally: Drake — Fake Love
✅ Found locally: Childish Gambino — Redbone
✅ Found locally: Khalid — Location
✅ Found locally: Khalid — Saved
✅ Found locally: Khalid — Talk (feat. Disclosure)

🎧 Final Rea

In [ ]:
# STEP 10: Compute mood distribution from predicted moods
mood_counts = final_df['predicted_mood'].value_counts(normalize=True).to_dict()

# STEP 11: Infer MBTI from mood distribution
mbti_type = infer_mbti_from_mood(mood_counts)

# STEP 12: Show MBTI result
print("\n🔮 Predicted MBTI Personality Type for Playlist:")
print(mbti_type)



🔮 Predicted MBTI Personality Type for Playlist:
INTJ


In [ ]:
import requests
import time
import pandas as pd

# 👇 Replace this with your desired playlist URL
new_playlist_url = "https://open.spotify.com/playlist/0nDIXUJbLjOEwPn1Idhz1r?si=1b6a65dbc0764ebb"

# 🎫 Get fresh Spotify token
token = get_token()  # Assumes get_token() is already defined

# 🎧 Step 1: Get songs from playlist
songs = get_playlist_tracks_only(new_playlist_url, token)  # Already defined function
print(f"Retrieved {len(songs)} songs.")

# 📝 Step 2: Fetch lyrics for each song using lyrics.ovh
lyrics_list = []
for song in songs:
    artist = song["artist"]
    title = song["title"]
    try:
        url = f"https://api.lyrics.ovh/v1/{artist}/{title}"
        response = requests.get(url)
        if response.status_code == 200:
            lyrics = response.json().get('lyrics', None)
        else:
            lyrics = None
        lyrics_list.append({
            "artist": artist,
            "title": title,
            "lyrics": lyrics
        })
        time.sleep(0.5)  # Be polite to the API
    except Exception as e:
        print(f"❌ Error fetching lyrics for {title} by {artist}: {e}")
        lyrics_list.append({
            "artist": artist,
            "title": title,
            "lyrics": None
        })

# ✅ Convert to DataFrame for BERT analysis
lyrics1_df = pd.DataFrame(lyrics_list)

# Drop entries with missing lyrics
lyrics_df = lyrics1_df.dropna(subset=['lyrics'])

print("\n📄 Lyrics fetched successfully:")
print(lyrics_df[['artist', 'title', 'lyrics']].head())


Retrieved 21 songs.

📄 Lyrics fetched successfully:
           artist                             title  \
5      Pink Floyd  Another Brick in the Wall, Pt. 2   
6   Guns N' Roses         Knockin' On Heaven's Door   
8   Guns N' Roses              Don't Cry (Original)   
9      Pink Floyd                              Time   
11  Ariana Grande                     One Last Time   

                                               lyrics  
5   We don't need no education\r\nWe don't need no...  
6   Mama take this badge from me\r\nI can't use it...  
8   Talk to me softly\r\nThere's something in your...  
9   Ticking away the moments that make up a dull d...  
11  I was a liar\r\nI gave into the fire\r\nI know...  


In [ ]:
# Make sure you have these loaded already
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(sentiment_encoder.classes_))
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# bert_model.to(device)
bert_model.eval()

from sklearn.preprocessing import LabelEncoder
import torch.nn.functional as F

# If not loaded yet:
label_encoder = joblib.load("bert_sentiment_model.pt")

# Process lyrics and predict sentiment using BERT only
lyrics_preds = []
lyrics_probs = []

for lyrics in lyrics1_df['lyrics']:
    if pd.isna(lyrics) or not isinstance(lyrics, str):
        lyrics_preds.append(None)
        lyrics_probs.append(None)
        continue

    try:
        inputs = tokenizer(lyrics, return_tensors="pt", truncation=True, padding=True, max_length=128).to(device)
        with torch.no_grad():
            outputs = bert_model(**inputs)
            logits = outputs.logits
            probs = F.softmax(logits, dim=1).cpu().numpy()[0]
            pred_idx = probs.argmax()
            pred_label = label_encoder.inverse_transform([pred_idx])[0]
            lyrics_preds.append(pred_label)
            lyrics_probs.append(probs)
    except Exception as e:
        print(f"⚠️ BERT failed on lyrics: {e}")
        lyrics_preds.append(None)
        lyrics_probs.append(None)

# Add to dataframe
lyrics1_df['bert_sentiment'] = lyrics_preds
lyrics1_df['bert_probs'] = lyrics_probs

# Show results
print("\n📝 BERT-Only Sentiment Predictions on Playlist:")
print(lyrics1_df[['artist', 'title', 'bert_sentiment']])


UnpicklingError: persistent IDs in protocol 0 must be ASCII strings

In [ ]:
torch.save(model.state_dict(), "bert_sentiment_model.pt")


In [ ]:
import joblib
joblib.dump(rf_model, "random_forest_mood.pkl")


In [ ]:
joblib.dump(fusion_model, "fusion_mlp_model.pkl")


In [ ]:
# Assuming fusion_model is your trained FusionNet instance
torch.save(fusion_model.state_dict(), "fusion_mlp_state_dict.pth")


In [ ]:
from google.colab import files
files.download("fusion_mlp_state_dict.pth")


In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Load dataset
df = pd.read_csv("spotify_2500_refined_moods (1).csv")
df = df.dropna(subset=['clean_lyrics', 'lyrics_sentiment', 'mood'])

# Clean and encode labels
df['lyrics_sentiment'] = df['lyrics_sentiment'].str.strip().str.lower()
df['mood'] = df['mood'].str.strip().str.lower()

sentiment_encoder = LabelEncoder()
df['sentiment_label'] = sentiment_encoder.fit_transform(df['lyrics_sentiment'])

mood_encoder = LabelEncoder()
df['mood_label'] = mood_encoder.fit_transform(df['mood'])

# Audio features
audio_features = ['valence', 'energy', 'danceability', 'tempo']

# Train/test split
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['mood_label'], random_state=42)

# Re-train the random forest on training set
rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
rf_model.fit(train_df[audio_features], train_df['mood_label'])

# Load trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(sentiment_encoder.classes_))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)
bert_model.eval()

# Function to get BERT sentiment probabilities
def get_bert_probs(texts):
    inputs = tokenizer(list(texts), padding=True, truncation=True, return_tensors="pt", max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = bert_model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1).cpu().numpy()
    return probs

# Build dataset of combined predictions
fusion_inputs = []
fusion_labels = []

for _, row in test_df.iterrows():
    audio_vec = rf_model.predict_proba([row[audio_features].values])[0]  # shape (8,)
    lyrics_vec = get_bert_probs([row['clean_lyrics']])[0]                # shape (5,)

    # Pad lyrics vector to 6 to match input_dim = 14
    if len(lyrics_vec) < 6:
        lyrics_vec = np.pad(lyrics_vec, (0, 6 - len(lyrics_vec)), mode='constant')

    combined_vec = np.concatenate([audio_vec, lyrics_vec])  # shape (14,)
    fusion_inputs.append(combined_vec)
    fusion_labels.append(row['mood_label'])

X_fusion = np.array(fusion_inputs)
y_fusion = np.array(fusion_labels)

# Dataset and DataLoader for FNN
class FusionDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = torch.tensor(inputs, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.inputs[idx], self.labels[idx]

fusion_dataset = FusionDataset(X_fusion, y_fusion)
fusion_loader = DataLoader(fusion_dataset, batch_size=16, shuffle=True)

# Define the FusionNet model
class FusionNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FusionNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

fusion_model = FusionNet(input_dim=14, hidden_dim=32, output_dim=len(mood_encoder.classes_))
fusion_model.to(device)

# TRAINING: FusionNet for 20 Epochs
optimizer = torch.optim.Adam(fusion_model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
epochs = 20

for epoch in range(epochs):
    fusion_model.train()
    total_loss, correct, total = 0, 0, 0
    for features, labels in fusion_loader:
        features, labels = features.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = fusion_model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    acc = correct / total
    avg_loss = total_loss / len(fusion_loader)
    print(f"Epoch {epoch+1:02d} | Loss: {avg_loss:.4f} | Accuracy: {acc:.4f}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feat

Epoch 01 | Loss: 1.9765 | Accuracy: 0.4016
Epoch 02 | Loss: 1.8942 | Accuracy: 0.4498
Epoch 03 | Loss: 1.8163 | Accuracy: 0.4538
Epoch 04 | Loss: 1.7345 | Accuracy: 0.4378
Epoch 05 | Loss: 1.6406 | Accuracy: 0.4458
Epoch 06 | Loss: 1.5527 | Accuracy: 0.4378
Epoch 07 | Loss: 1.4925 | Accuracy: 0.4498
Epoch 08 | Loss: 1.4033 | Accuracy: 0.4538
Epoch 09 | Loss: 1.3321 | Accuracy: 0.4940
Epoch 10 | Loss: 1.2852 | Accuracy: 0.5341
Epoch 11 | Loss: 1.1962 | Accuracy: 0.5984
Epoch 12 | Loss: 1.1533 | Accuracy: 0.6546
Epoch 13 | Loss: 1.0846 | Accuracy: 0.6707
Epoch 14 | Loss: 1.0022 | Accuracy: 0.7229
Epoch 15 | Loss: 0.9604 | Accuracy: 0.7510
Epoch 16 | Loss: 0.9159 | Accuracy: 0.7550
Epoch 17 | Loss: 0.8942 | Accuracy: 0.7671
Epoch 18 | Loss: 0.8201 | Accuracy: 0.7912
Epoch 19 | Loss: 0.8066 | Accuracy: 0.8032
Epoch 20 | Loss: 0.7543 | Accuracy: 0.7952


In [ ]:
# Save
torch.save(fusion_model.state_dict(), 'fusion_model1.pth')